# 이미지 사이즈 옵션 정리
- 1. (yolo - optional) 416 -> (efficientnet) 224 (preferred)
- 2. (yolo - optional) 620 -> (efficientnet) 400

`실제 이미지셋 width, height 분석 결과 `
- width의 평균값 :  382
- height의 평균값 :  430

# 구글 드라이브 연동하기

In [ ]:
# 구글 드라이브 연동하기
from google.colab import drive
drive.mount('/content/drive')

# 경로 및 파일 관리
import os

# change directory 경로 변경하기
print(os.chdir('/content/drive/MyDrive'))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
None


In [ ]:
# # yolo5 파일에서 필요한 것을 설치하기 위한 경로 설정하기
# os.chdir('/content/drive/MyDrive/yolo_efficientnet/yolov5')

# # 환경 설정 파일 설치 -> [변경 내용] requests==2.31.0   pillow<10.1.0
# %pip install -qr requirements.txt

# # 공유 데이터셋 파일을 가져오기 위한 roboflow
# %pip install -q roboflow

# # 딥러닝 라이브러리
# import torch

# # yaml : json 또는 xml 처럼 데이터 송수신을 위해 사전에 정해진 규칙을 따르는 '데이터 전송 파일' 형식
# import yaml

# # 이미지 보여주는 용도
# from PIL import Image as Img
# from IPython.display import Image, display # clear_output

# # 특정 확장자 파일 불러오기 -> glob.glob() : 특정 패턴과 일치하는 모든 경로명을 리스트로 반환
# import glob

# # 학습한 베스트 모델 다운로드하기
# from google.colab import files

### import

In [ ]:
!pip install extcolors

In [ ]:
# 딥러닝 사용
import torch

# 특정 확장자 파일 불러오기 -> glob.glob() : 특정 패턴과 일치하는 모든 경로명을 리스트로 반환
import glob

# 이미지 관련
from PIL import Image as Img
from PIL import ImageFile
import cv2
import numpy as np
import glob
from IPython.display import Image, display

# 문자열 처리
import unicodedata

# 색 추출
import extcolors


# 파일 관리
import os

# efficientNet_v2_s 모델을 가지고오기 위함
import torchvision.models as get_model
import torchvision

# input 데이터를 정규화하기 위한 transform 규칙을 적용하기 위함
import torchvision.transforms as transforms

# optimizer를 가지고 오기 위함.
import torch.optim as optim

# pytorch에서 지원하는 다양한 계층 및 손실함수 계산을 사용하기 위함.
import torch.nn as nn

from torchvision.datasets import ImageFolder
from torchvision import transforms # ToTensor, Resize, Compose
from torch.utils.data import DataLoader
from torch.utils.data import Dataset, DataLoader

# 스케쥴러
from torch.optim.lr_scheduler import OneCycleLR

# 학습한 베스트 모델 다운로드하기
from google.colab import files



### 이미지 비율을 유지하면서 resize하는 함수 정의

In [ ]:
def maintain_proportion_and_resize_by_cv2(image_path, size=(416, 416)):

    # 이미지 읽기(BGR)
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"이미지를 열 수 없습니다: {image_path}")

    # 이미지 형식 변경하기(BGR -> RGB)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 높이, 너비 추출
    original_height, original_width = image.shape[:2]

    # 비율 계산
    ratio = min(size[0] / original_width, size[1] / original_height)
    new_width = int(original_width * ratio)
    new_height = int(original_height * ratio)

    # 이미지 리사이즈
    resized_image = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_AREA)

    # 새로운 이미지를 패딩하여 416x416 크기로 만들기
    new_image = np.full((size[1], size[0], 3), (255, 255, 255), dtype=np.uint8)  # 흰색 배경으로 초기화
    x_offset = (size[0] - new_width) // 2
    y_offset = (size[1] - new_height) // 2
    new_image[y_offset:y_offset + new_height, x_offset:x_offset + new_width] = resized_image

    return new_image


def maintain_proportion_and_resize_by_pil(image, size=(416, 416)):

    # PIL.Image 형식의 이미지를 비율을 유지하면서 지정된 크기로 리사이즈하고,
    # 흰색 배경으로 패딩하여 최종 이미지를 반환

    # 원본 이미지 크기 추출
    original_width, original_height = image.size

    # 비율 계산
    ratio = min(size[0] / original_width, size[1] / original_height)
    new_width = int(original_width * ratio)
    new_height = int(original_height * ratio)

    # 이미지 리사이즈
    resized_image = image.resize((new_width, new_height), Img.ANTIALIAS)

    # 새로운 이미지를 패딩하여 지정된 크기로 만들기
    new_image = Img.new("RGB", size, (255, 255, 255))  # 흰색 배경으로 초기화
    x_offset = (size[0] - new_width) // 2
    y_offset = (size[1] - new_height) // 2
    new_image.paste(resized_image, (x_offset, y_offset))

    return new_image

In [ ]:
device = torch.device('cuda'if torch.cuda.is_available else 'cpu')

### 학습된 yolo 모델 불러오기

In [ ]:
# yolov5 pt 파일 저장 이름 정의
yolov5_best_pt_save_name = 'final_fashion_yolo_best'

# yolo pt 파일 저장 경로 정의
yolov5_best_pt_save_path = f'/content/drive/MyDrive/yolo_efficientnet/yolo_best.pt/{yolov5_best_pt_save_name}.pt'

# 학습 가중치를 적용한 모델 불러오기
yolov5_model = torch.hub.load('ultralytics/yolov5', 'custom', path= yolov5_best_pt_save_path)

# device 설정
yolov5_model.to(device)

# yolov5_model 평가 모드 선언
yolov5_model.eval()

### 학습된 EfficientNet 모델 불러오기

In [ ]:
# 학습되지 않은 초기화 efficientnetv2_s 모델 불러오기
effnet_v2_s_model = get_model.efficientnet_v2_s(pretrained=False)


# 학습된 pt 파일의 out_features 개수
num_classes = 38

# out_features 클래스 개수 맞춰 주기
in_features = effnet_v2_s_model.classifier[1].in_features
effnet_v2_s_model.classifier[1] = nn.Linear(in_features, num_classes)

# 반영 여부 확인
print('pt 적용 전 계층 구조 맞추기')
print(effnet_v2_s_model.classifier)

pt 적용 전 계층 구조 맞추기
Sequential(
  (0): Dropout(p=0.2, inplace=True)
  (1): Linear(in_features=1280, out_features=38, bias=True)
)


In [ ]:
# 이전까지 학습했던 파라미터 파일 적용하기

eff_pt_root_path = '/content/drive/MyDrive/yolo_efficientnet/efficientnet_train.pt/'
eff_pt_name = f'trained_{260}_effnet.pt'

checkpoint = torch.load(eff_pt_root_path + eff_pt_name)

# Load the model state dictionary
effnet_v2_s_model.load_state_dict(checkpoint['model_state_dict'])

# device 설정
effnet_v2_s_model.to(device)

# 평가 모드 선언
effnet_v2_s_model.eval()

# 계층 적용 여부 점검 -> out_features : 38개여야 됨.
print('pt 적용 후 계층 구조 확인')
print(effnet_v2_s_model.classifier)

pt 적용 후 계층 구조 확인
Sequential(
  (0): Dropout(p=0.2, inplace=True)
  (1): Linear(in_features=1280, out_features=38, bias=True)
)


### yolo 모델 + efficientnet 모델 통합 로직

- 1. 하나의 이미지가 yolo에 들어간다 -> bounding_box(들)을 추출한다. [`원본 이미지`✅, `bounding box(들)`✅]

- 2. bounding_box(들)에서 주요 옷 색상을 추출한다. [`색상 정보 추출`✅]

- 3. bounding_box(들)에서 대분류 정보를 추출한다. [`대분류 정보`✅]


In [ ]:
color_ranges = {
    "red": ((128, 0, 0), (255, 127, 127)),
    "green": ((0, 128, 0), (127, 255, 127)),
    "blue": ((0, 0, 128), (127, 127, 255)),
    "yellow": ((128, 128, 0), (255, 255, 127)),
    "magenta": ((128, 0, 128), (255, 127, 255)),
    "cyan": ((0, 128, 128), (127, 255, 255)),
    "orange": ((128, 64, 0), (255, 191, 127)),
    "purple": ((64, 0, 64), (191, 127, 191)),
    "pink": ((255, 0, 127), (255, 191, 255)),
    "lime": ((0, 255, 0), (191, 255, 191)),
    "brown": ((64, 32, 0), (191, 127, 64)),
    "gray": ((128, 128, 128), (191, 191, 191)),
    "black": ((0, 0, 0), (63, 63, 63)),
    "white": ((192, 192, 192), (255, 255, 255))
}


def get_color_name(rgb_value):
    for color_name, (lower_bound, upper_bound) in color_ranges.items():
        # 특정 색깔 범위에 있으면
        if all(lower_bound[i] <= rgb_value[i] <= upper_bound[i] for i in range(3)):
            return color_name
    return "UNKNOWN"

In [ ]:
# 문자열 처리
def normalize(text) :
    normalized_text = unicodedata.normalize('NFD', text)
    return normalized_text

# 클래스 라벨 사전 정의하기
classes_composition_list = ['니트','후드','맨투맨','셔츠블라우스','긴소매티셔츠','반소매티셔츠','민소매티셔츠','카라티셔츠','베스트','데님팬츠','슬랙스','트레이닝조거팬츠','숏팬츠','코튼팬츠','레깅스','와이드팬츠','후드집업','바람막이',
'코트','롱패딩','숏패딩','패딩베스트','블루종','레더자켓','무스탕','트러커자켓','블레이저','가디건','뽀글이후리스','사파리자켓','미니원피스','미디원피스','투피스','롱원피스', '점프수트','미니스커트','미디스커트','롱스커트']

cls_labels = {normalize(cls) : i for i, cls in enumerate(classes_composition_list)}

cls_labels = {v:k for k, v in cls_labels.items()}

# 하나의 이미지 경로를 넣으면 상위, 하위 카테고리를 추출해주는 함수 정의

In [ ]:
# yolo 대분류 정의
main_category = {'top' : '상의','bottom' : '바지','outer' : '아우터','skirt' : '스커트','dress' : '원피스'}

# 대분류, 소분류 맵핑
categories = {
    '상의' : ['니트', '후드', '맨투맨','셔츠블라우스','긴소매티셔츠','반소매티셔츠','민소매티셔츠','카라티셔츠','베스트'],
    '바지' : ['데님팬츠', '슬랙스','트레이닝조거팬츠','숏팬츠','코튼팬츠','레깅스', '와이드팬츠'],
    '아우터' : ['후드집업', '바람막이', '코트', '롱패딩', '숏패딩', '패딩베스트', '블루종',
                '레더자켓', '무스탕', '트러커자켓', '블레이저', '가디건','뽀글이후리스','사파리자켓'],
    '원피스' : ['미니원피스', '미디원피스', '투피스', '롱원피스', '점프수트'],
    '스커트' : ['미니스커트', '미디스커트', '롱스커트']
}

In [ ]:
def extract_main_sub_category(image_path) :
  # 바운딩 박스 집합 배열
  detected_image_composition_list = []
  mains = []
  subs = []

  # 깔끔한 bbox 추출 사진을 위한 준비
  resized_img_for_cropping = maintain_proportion_and_resize_by_cv2(test_image_path, (416, 416))

  # yolo input : 416X416 resize 하기, 비율 유지하기
  resized_img = maintain_proportion_and_resize_by_cv2(test_image_path, (416, 416))

  # 모델 투입-> 결과 도출
  result = yolov5_model(resized_img)

  # bounding box 탐지 실패한 경우
  if not len(result.xyxy[0]) :
    return ['탐지 실패']

  # 탐지 성공한 경우
  for i, bounding_box in enumerate(result.xyxy[0]):
    # 바운딩 박스 좌표 정의
    xmin, ymin, xmax, ymax = map(int, bounding_box[:4])
    # 신뢰도 정의
    confidence = round(bounding_box[4].item(), 2)
    # 탐지 클래스 정의
    cls = int(bounding_box[5])
    class_name = result.names[cls]

    # bounding box 보여주기
    cropped_img = resized_img_for_cropping[ymin:ymax, xmin:xmax]
    cropped_img = Img.fromarray(cropped_img, 'RGB')

    # 바운딩 박스 모으기
    detected_image_composition_list.append(cropped_img)
    mains.append(main_category[class_name])

  for detected_image in detected_image_composition_list :
    resized_image = maintain_proportion_and_resize_by_pil(detected_image, (226, 226))

    # display(resized_image)

    resized_image = image_to_tensor(resized_image)
    resized_imgae = resized_image.unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = effnet_v2_s_model(resized_imgae)

    max_value, max_index  = torch.max(outputs, 1)
    # [소분류 정보✅]
    subs.append(cls_labels[max_index.item()])

  filtered_bbox = [detected_image_composition_list[i] for i in range(len(result.xyxy[0])) if subs[i] in categories[mains[i]]]
  filtered_main_sub = [f'{mains[i]}_{subs[i]}' for i in range(len(result.xyxy[0])) if subs[i] in categories[mains[i]]]

  # 모든 것의 맵핑 결과 맞지 않으면
  if not len(filtered_bbox) :
    return ['맵핑 실패']

  return (resized_img, filtered_bbox, filtered_main_sub)

In [ ]:
# 이미지를 텐서로 변환하기
image_to_tensor = transforms.Compose([
    transforms.ToTensor()])

# 테스트 이미지 경로 정의 -> 학습 했던 내용을 기반으로 해보자!--> 결과가 괜찮게 나올수도?
image_dataset_to_test_path = "/content/drive/MyDrive/yolo_efficientnet/datasets_for_unlabeled/"

# 탐지된 바운딩 박스를 모을 리스트
detected_image_composition_list = []

for test_image in os.listdir(image_dataset_to_test_path)[1200:1230] :

    test_image_path = image_dataset_to_test_path + test_image

    # natural_image 만들기
    natural_img = cv2.imread(test_image_path)
    natural_img = cv2.cvtColor(natural_img, cv2.COLOR_BGR2RGB)
    natural_img = Img.fromarray(natural_img)

    # # [원본 이미지✅] 화면출력
    # display(natural_img)

    # yolo에 들어가기 위해서 416X416 사이즈로 비율을 고려하여 resize 하기
    resized_img = maintain_proportion_and_resize_by_cv2(test_image_path, (416, 416))

    # 깔끔한 추출 사진을 얻기 위해서 정의
    resized_img_for_cropping = maintain_proportion_and_resize_by_cv2(test_image_path, (416, 416))

    # resize한 사진을 모델에 넣고
    result = yolov5_model(resized_img)

    # 탐지한 것이 있다면
    if len(result.xyxy[0]) != 0:
        print(f'{test_image}의 탐지 결과 :{len(result.xyxy[0])}개 ')

    else :
        print(f'{test_image}의 탐지 결과 없음')

    for i, bounding_box in enumerate(result.xyxy[0]):
        xmin, ymin, xmax, ymax = map(int, bounding_box[:4])

        # [대분류 정보✅]
        cls = int(bounding_box[5])
        class_name = result.names[cls]

        # 신뢰도 측정
        confidence = round(bounding_box[4].item(), 2)

        # bounding box 보여주기
        cropped_img = resized_img_for_cropping[ymin:ymax, xmin:xmax]
        cropped_img = Img.fromarray(cropped_img, 'RGB')

        # 바운딩 박스 모으기
        detected_image_composition_list.append(cropped_img)

        # [resize된 bounding box(들)✅]
        display(maintain_proportion_and_resize_by_pil(cropped_img, (224, 224)))

        print(main_category[class_name])

    print('_________________________________________________________')





Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# bounding box 모은 것을 반복문 돌리면서 resize 하기 -> (224, 224)
# [resize 이미지✅]
for detected_image in detected_image_composition_list :
    resized_image = maintain_proportion_and_resize_by_pil(detected_image, (224, 224))

    display(resized_image)

    resized_image = image_to_tensor(resized_image)
    resized_imgae = resized_image.unsqueeze(0).to(device)



    with torch.no_grad():
        outputs = effnet_v2_s_model(resized_imgae)

    max_value, max_index  = torch.max(outputs, 1)
    # [소분류 정보✅]
    print(cls_labels[max_index.item()])

Output hidden; open in https://colab.research.google.com to view.